In this notebook, we will cover topics related to regular expressions:

* [Basic matches](#Basic-matches)
* [Anchors](#Anchors)
* [Character classes](#Character-classes)
* [Alternatives](#Alternatives)

# Basic matches

In [1]:
library(tidyverse)
library(stringr)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



We looked at the `str_c` function last time that is used to concatenate strings (and vectors of strings). The opposite of concatenating is splitting, a job done by the `str_split` function.

In [2]:
fruits <- c("an apple, a banana, a cherry", "a kiwifruit, a lemon, a mango")
(after_split <- str_split(fruits, ", "))

[[1]]
[1] "an apple" "a banana" "a cherry"

[[2]]
[1] "a kiwifruit" "a lemon"     "a mango"

In [3]:
# str_split returns a list of vectors, one vector for each string you're splitting
# here we're just splitting one string so we get a list with one element that we extract using [[1]]
(after_split <- str_split("an apple, a banana, a cherry", ", ")[[1]])
# str_c will undo the split
str_c(after_split, collapse=", ")

[1] "an apple" "a banana" "a cherry"

[1] "an apple, a banana, a cherry"

In [4]:
str_split('this/is/a/unix/style/file/path','/')

[[1]]
[1] "this"  "is"    "a"     "unix"  "style" "file"  "path"

In [5]:
path <- 'this\\is\\a\\windows\\style\\file\\path'
writeLines(path)

this\is\a\windows\style\file\path


In [6]:
str_split(path, '\\')

ERROR: ignored

Why does the command not work? If you look up the documentation of `str_split()`, you will find that that the pattern given for splitting is interpreted as a **regular expression**. A regular expression is a way to describe a systematic pattern in strings. The simplest pattern is simple a given, fixed string.

In [7]:
x <- c("statistics", "data science", "machine learning")
str_detect(x, "at") # search for the pattern 'at'

[1]  TRUE  TRUE FALSE

In [8]:
str_extract(x, "at") 

[1] "at" "at" NA

In [9]:
str_detect(x, ".i.") # . matches any single character except newline

[1] TRUE TRUE TRUE

In [10]:
str_extract(x, ".i.")

[1] "tis" "cie" "hin"

In [11]:
str_extract_all(x, ".i.") # extracts all matches, not just the first one

[[1]]
[1] "tis" "tic"

[[2]]
[1] "cie"

[[3]]
[1] "hin" "nin"

In [12]:
mystring = "A . string . with . periods ..."

In [13]:
str_detect(mystring, ".") # doesn't match the period since . has a special meaning in regexps

[1] TRUE

In [14]:
str_detect(mystring, "\.") # why does this not work?

ERROR: ignored

In [15]:
str_detect(mystring, "\\.") # why do we need double backslashes?

[1] TRUE

In [16]:
str_extract_all(mystring, "\\.")

[[1]]
[1] "." "." "." "." "." "."

In [17]:
mystring2 <- "A \\ string \\ with \\ backslashes \\ \\ \\"
writeLines(mystring2)

A \ string \ with \ backslashes \ \ \


In [18]:
str_extract_all(mystring2, "\\") # doesn't work

ERROR: ignored

In [19]:
str_extract_all(mystring2, "\\\\") # Gosh -- need 4 backslashes to match a single backslash!!!

[[1]]
[1] "\\" "\\" "\\" "\\" "\\" "\\"

Now we know how to fix the problem with the `str_split()` code above.

In [20]:
writeLines(path)
str_split(path, '\\\\')

this\is\a\windows\style\file\path


[[1]]
[1] "this"    "is"      "a"       "windows" "style"   "file"    "path"

In [21]:
x <- c("a.b.c.d", ".a.b.", "a.b.", ".1.1.1", ". . . ", "...abc")

We want to match the pattern:

`<period> <any char> <period> <any char> <period> <any char>`

The regular expression for this is:

`\..\..\..`

So the string representing the regular expression is:

`"\\..\\..\\.."`

In [22]:
re <- "\\..\\..\\.."
str_detect(x, re)

[1]  TRUE FALSE FALSE  TRUE  TRUE FALSE

# Anchors

In [23]:
x <- c("statistics", "data science", "machine learning")
str_detect(x, "s")

[1]  TRUE  TRUE FALSE

In [24]:
str_detect(x, "^s") # ^ matches the beginning of a string

[1]  TRUE FALSE FALSE

In [25]:
str_detect(x, "s$") # $ matches the ending of a string

[1]  TRUE FALSE FALSE

In [26]:
y <- c("earn $$$", "cost is $10", "nothing to do with money!")
str_detect(y, "$") # not the write way to match a literal $ sign

[1] TRUE TRUE TRUE

In [27]:
str_detect(y, "\\$")

[1]  TRUE  TRUE FALSE

In [28]:
str_extract_all(y, "\\$")

[[1]]
[1] "$" "$" "$"

[[2]]
[1] "$"

[[3]]
character(0)

# Character classes

These special patterns match characters in a class:

* `\d`: matches any digit.
* `\s`: matches any whitespace (e.g. space, tab, newline).
* `[abc]`: matches a, b, or c.
* `[^abc]`: matches anything *except* a, b, or c.

In [29]:
x <- c("1", "one", "23", "twothree", "r2d2")
str_detect(x, "\\d") # why double backslash?

[1]  TRUE FALSE  TRUE FALSE  TRUE

In [30]:
x <- c("one", "two", "three", "four", "five")
vowel_re <- "[aeiou]"
str_extract_all(x, vowel_re)

[[1]]
[1] "o" "e"

[[2]]
[1] "o"

[[3]]
[1] "e" "e"

[[4]]
[1] "o" "u"

[[5]]
[1] "i" "e"

# Alternatives

Alternative patterns can be matched using `|`.

In [31]:
color_re = "colo(r|ur)"
x <- c("color", "red colour", "coloured glass", "chair", "colored chair")
str_detect(x, color_re)

[1]  TRUE  TRUE  TRUE FALSE  TRUE

Suppose we want to match telephone numbers of the form:

* xxx-xxx-xxxx
* (xxx) xxx-xxxx

In [32]:
phone_re = "(\\d\\d\\d-|\\(\\d\\d\\d\\) )\\d\\d\\d-\\d\\d\\d\\d" # complicated because of all the double backslashes

In [33]:
n <- c("123-456-7890", "(123) 456-7890", "1234567890", "+1-123-456-7890")
str_detect(n, phone_re)

[1]  TRUE  TRUE FALSE  TRUE